# Model Free Control
 
Sungchul Lee 

# How to run these slides yourself

**Setup python environment**

- [Install RISE for an interactive presentation viewer](https://github.com/damianavila/RISE)

# How to learn $Q_*$

\begin{array}{ccc}\hline
\mbox{Prediction (on-policy)}&\mbox{Control (on-policy)}&\mbox{Control (off-policy)}\\\hline
\mbox{MC}&\mbox{MC}\\
\mbox{TD}&\mbox{SARSA}&\mbox{Q-learnig}\\
\mbox{TD($\lambda$)}&\mbox{SARSA($\lambda$)}\\\hline
\end{array}

# SARSA 





周莫烦
[youtube](https://www.youtube.com/watch?v=2-OljVM15gc)
[github](https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow)
Silver
[4](https://www.youtube.com/watch?v=PnHCvfgC_ZA&list=PL7-jPKtc4r78-wCZcQn5IqyuWhBZ8fOxT&index=4) 
[pdf](http://localhost:8888/notebooks/Dropbox/Paper/Reinforcement_Learning_by_David_Silver_4.pdf)
[5](https://www.youtube.com/watch?v=0g4j2k_Ggc4&index=5&list=PL7-jPKtc4r78-wCZcQn5IqyuWhBZ8fOxT) 
[pdf](http://localhost:8888/notebooks/Dropbox/Paper/Reinforcement_Learning_by_David_Silver_5.pdf)
Huang [1](https://towardsdatascience.com/introduction-to-various-reinforcement-learning-algorithms-i-q-learning-sarsa-dqn-ddpg-72a5e0cb6287) [2](https://towardsdatascience.com/introduction-to-various-reinforcement-learning-algorithms-part-ii-trpo-ppo-87f2c5919bb9)
BENNETT
[blog](https://articles.wearepop.com/secret-formula-for-self-learning-computers)

##### $\varepsilon$-greedy policy update
- Update $\varepsilon$-greedy policy using udated $Q$
$$
\pi(a|s)=\left\{\begin{array}{ll}
\frac{\varepsilon}{m}+(1-\varepsilon)&\mbox{if}\ a=\mbox{argmax}_{a'}Q(s,a')\\
\frac{\varepsilon}{m}&\mbox{otherwise}
\end{array}\right.
$$
where $m$ is the number of actions.
- Choose action using updated $\varepsilon$-greedy policy 

##### SARSA (On-Policy) update
With $a_{t+1}$ from the data
$$
Q(s_t,a_t)\quad\leftarrow\quad
Q(s_t,a_t)+\alpha(\color{red}{r_{t+1}+\gamma Q(s_{t+1},a_{t+1})}-Q(s_t,a_t))
$$


<div align="center"><img src="img/RZBt6.png" width="100%" height="30%"></div>

https://i.stack.imgur.com/RZBt6.png

<div align="center"><img src="img/Screen Shot 2017-11-24 at 5.49.55 PM.png" width="100%" height="30%"></div>


In [3]:
# SARSA

# import libraries
import numpy as np

# set parameters ###############################################################
epoch = 1000
gamma = 0.99
alpha = 0.01
epsilon = 0.01
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3]  # left, right, up, down
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 1: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 1: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))

# Q
Q = np.zeros((N_STATES, N_ACTIONS))
Q[3, :] = 1.
Q[6, :] = -1.

# SARSA
for _ in range(epoch):

    # indicate game is not over yet
    done = False
    
    # choose initial state randomly, not from 3 or 6
    s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10]) 
    
    # this is policy update part of algorithm 
    # epsilon-greedy policy update
    # choose action using udated epsilon-greedy policy
    # if this part is replaced and if we choose action using current policy
    # this whole algorithm becomes TD, not SARSA
    if np.random.uniform(0.,1.) < epsilon:
        # random action selection
        a = np.random.choice(actions)
    else:
        # greedy action selection
        a = np.argmax(Q[s, :])

    while not done:
        # choose next state using transition probabilities
        s1 = np.random.choice(states, p=P[s, a, :])

        # this is policy update part of algorithm 
        # epsilon-greedy policy update
        # choose action using udated epsilon-greedy policy
        # if this part is replaced and if we choose action using current policy
        # this whole algorithm becomes TD, not SARSA
        if np.random.uniform(0.,1.) < epsilon:
            # random action selection
            a1 = np.random.choice(actions)
        else:
            # greedy action selection
            a1 = np.argmax(Q[s1, :])

        # this is policy estimate part of algorithm 
        # SARSA
        Q[s, a] = Q[s, a] + alpha * (R[s, a] + gamma * Q[s1, a1] - Q[s, a])

        if (s1 == 3) or (s1 == 6):
            # if game is over, break while loop 
            done = True
        else:
            # if game is not over, continue playing game
            s = s1
            a = a1

print(Q)

[[ 0.17272066  0.84813053  0.21954704  0.19589769]
 [ 0.28294384  0.89666282  0.30808414  0.26875633]
 [ 0.55679543  0.92789155  0.60644865  0.44249506]
 [ 1.          1.          1.          1.        ]
 [ 0.11626333  0.10769002  0.80749392  0.08102746]
 [ 0.56449172 -0.50744998  0.64892495  0.33351357]
 [-1.         -1.         -1.         -1.        ]
 [ 0.0346382   0.58767382  0.11264302  0.03821519]
 [ 0.1442668   0.59643485  0.12966441  0.1490232 ]
 [ 0.20848015  0.21009526  0.61976415  0.25303205]
 [ 0.45106223  0.00676568 -0.18431103  0.00751391]]


# Convergence of SARSA - GLIE and Robins-Monro



<div align="center"><img src="img/GLIE.png" width="80%" height="30%"></div>

http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/control.pdf

<div align="center"><img src="img/Convergence of Sarsa.png" width="80%" height="30%"></div>

http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/control.pdf

# Exercise

- In the SARSA code $\varepsilon$ stays as a constant during the $t$ iteration.
Modify the code and decrease $\varepsilon$ values as $t$ increases.
More specifically, as the theory sugests, modify the code so that
$$\varepsilon=\varepsilon(t)\sim\frac{1}{t}$$

- In the SARSA code $\alpha$ stays as a constant during the $t$ iteration.
Modify the code and decrease $\alpha$ values as $t$ increases.
More specifically, as the theory sugests, modify the code so that
$$\alpha=\alpha(t)\sim\frac{1}{t}$$

# SARSA($\lambda$) 

### SARSA update
$$
Q(s_t,a_t)\quad\leftarrow\quad
Q(s_t,a_t)+\alpha\delta_t
$$
- for each step SARSA error $\delta_t=r_{t+1}+\gamma Q(s_{t+1},a_{t+1})$ update only one $Q(s,a)$, i.e., $Q(s_t,a_t)$

### SARSA($\lambda$) update
$$\begin{array}{lll}
E_0(s,a)&=&0\\
E_t(s,a)&=&\gamma\lambda E_{t-1}(s,a)+1(S_t=s,A_t=a)\\
Q(s,a)&\leftarrow&Q(s,a)+\alpha\delta_t \color{red}{E_t(s,a)}
\end{array}$$
- $E_t(s,a)$ is called Eligibility Trace
- for each step SARSA error $\delta_t=r_{t+1}+\gamma Q(s_{t+1},a_{t+1})$ update all $Q(s,a)$ using eligibility trace $E_t(s,a)$

In [5]:
# SARSA(lambda)

# import libraries
import numpy as np

# set parameters ###############################################################
epoch = 40000
gamma = 0.99
alpha = 0.01
epsilon = 0.01
la = 0.1
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3]  # left, right, up, down
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 1: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 1: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))

" SARSA(lambda) "

# Q
Q = np.zeros((N_STATES, N_ACTIONS))
Q[3, :] = 1.
Q[6, :] = -1.

# eligibility trace
# E_0(s,a)=0
E =  np.zeros((N_STATES, N_ACTIONS)) 

for _ in range(epoch):

    # indicate game is not over yet
    done = False
    
    # choose initial state randomly, not from 3 or 6
    s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10]) 
    
    # this is policy update part of algorithm 
    # epsilon-greedy policy update
    # choose action using udated epsilon-greedy policy
    # if this part is replaced and if we choose action using current policy
    # this whole algorithm becomes TD, not SARSA
    if np.random.uniform(0.,1.) < epsilon:
        # random action selection
        a = np.random.choice(actions)
    else:
        # greedy action selection
        a = np.argmax(Q[s, :])
        
    # E_t(s2,a2) = \gamma\lambda E_{t-1}(s2,a2)+1(s2=s,a2=a)    
    E = gamma * la * E
    E[s,a] += 1.

    while not done:
        # choose next state using transition probabilities
        s1 = np.random.choice(states, p=P[s, a, :])

        # this is policy update part of algorithm 
        # epsilon-greedy policy update
        # choose action using udated epsilon-greedy policy
        # if this part is replaced and if we choose action using current policy
        # this whole algorithm becomes TD, not SARSA
        if np.random.uniform(0.,1.) < epsilon:
            # random action selection
            a1 = np.random.choice(actions)
        else:
            # greedy action selection
            a1 = np.argmax(Q[s1, :])

        # this is policy estimate part of algorithm 
        # SARSA
        # Q[s, a] = Q[s, a] + alpha * (R[s, a] + gamma * Q[s1, a1] - Q[s, a])
        # SARSA(lambda)
        delta = R[s,a] + gamma * Q[s1,a1] - Q[s,a]
        for s2 in states:
            if (s2!=3) and (s2!=6):
                for a2 in actions:
                    Q[s2,a2] = Q[s2,a2] + alpha * delta * E[s2,a2]

        if (s1 == 3) or (s1 == 6):
            # if game is over, break while loop 
            done = True
        else:
            # if game is not over, continue playing game
            s = s1
            a = a1
            
            # E_t(s2,a2) = \gamma\lambda E_{t-1}(s2,a2)+1(s2=s,a2=a)    
            E = gamma * la * E
            E[s,a] += 1.

print(Q)

[[ 0.18906137  0.84469577  0.20435256  0.21712334]
 [ 0.24753696  0.90113342  0.35720888  0.22884378]
 [ 0.54990844  0.94142078  0.61811901  0.49485598]
 [ 1.          1.          1.          1.        ]
 [ 0.10508289  0.12383426  0.79892224  0.08595944]
 [ 0.55304415 -0.49665848  0.64367556  0.34241047]
 [-1.         -1.         -1.         -1.        ]
 [ 0.0440567   0.56760054  0.082329    0.01777154]
 [ 0.13420097  0.59548236  0.1574516   0.16509338]
 [ 0.22459865  0.18931261  0.61539511  0.2404944 ]
 [ 0.47180639  0.03426495 -0.20771631  0.04800779]]


# Q-learning 

Raval
[youtube](https://www.youtube.com/watch?v=A5eihauRQvo)
[github](https://github.com/llSourcell/q_learning_demo)
[youtube](https://www.youtube.com/watch?v=A5eihauRQvo&t=91s)

### $\varepsilon$-greedy policy update
- Update $\varepsilon$-greedy policy using udated $Q$
$$
\pi(a|s)=\left\{\begin{array}{ll}
\frac{\varepsilon}{m}+(1-\varepsilon)&\mbox{if}\ a=\mbox{argmax}_{a'}Q(s,a')\\
\frac{\varepsilon}{m}&\mbox{otherwise}
\end{array}\right.
$$
where $m$ is the number of actions.
- Choose action using updated $\varepsilon$-greedy policy

### Q_learning  (Off-Policy) update
- [SARSA] With a sampling $a'$ from the data or data generating policy 
$$
Q(s_t,a_t)\quad\leftarrow\quad
Q(s_t,a_t)+\alpha(\color{red}{r_{t+1}+\gamma Q(s_{t+1},a')}-Q(s_t,a_t))
$$
- [Q-Learning] With a sampling $a'$ from the policy of interest, not from the data or data generating policy
$$
Q(s_t,a_t)\quad\leftarrow\quad
Q(s_t,a_t)+\alpha(\color{red}{r_{t+1}+\gamma Q(s_{t+1},a')}-Q(s_t,a_t))
$$
- [Q-Learning] If the policy of interest is greedy,
$$
Q(s_t,a_t)\quad\leftarrow\quad
Q(s_t,a_t)+\alpha(\color{red}{r_{t+1}+\gamma \max_{a'}Q(s_{t+1},a')}-Q(s_t,a_t))
$$

<div align="center"><img src="img/Images_Algorithm_pt2_3.gif" width="80%" height="30%"></div>

https://articles.wearepop.com/secret-formula-for-self-learning-computers

<div align="center"><img src="img/JvJqR.png" width="100%" height="30%"></div>

https://i.stack.imgur.com/JvJqR.png

<div align="center"><img src="img/Screen Shot 2017-11-24 at 5.55.31 PM.png" width="100%" height="30%"></div>

In [2]:
# Q-Learning

# import libraries
import numpy as np

# set parameters ###############################################################
epoch = 40000
gamma = 0.99
alpha = 0.01
epsilon = 0.01
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3]  # left, right, up, down
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

# policy
if 0: 
    # bad policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif 1: 
    # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif 1: 
    # optimal policy 
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
elif 1: 
    # optimal policy + noise 
    # we use optimal policy with probability 1/(1+ep)
    # we use random policy with probability ep/(1+ep)
    ep = 0.1
    policy = np.empty((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]
    policy = policy + (ep/4)*np.ones((N_STATES, N_ACTIONS))
    policy = policy / np.sum(policy, axis=1).reshape((N_STATES,1))

# Q
Q = np.zeros((N_STATES, N_ACTIONS))
Q[3, :] = 1.
Q[6, :] = -1.

# Q-Learning
for _ in range(epoch):
    
    # indicate game is not over yet
    done = False
    
    # choose initial state randomly, not from 3 or 6
    s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10]) 
    
    # this is policy update part of algorithm 
    # epsilon-greedy policy update
    # choose action using udated epsilon-greedy policy
    # if this part is replaced and if we choose action using current policy
    # this whole algorithm becomes TD, neither SARSA nor Q-Learning
    if np.random.uniform(0.,1.) < epsilon:
        a = np.random.choice(actions)
    else:
        a = np.argmax(Q[s, :])

    while not done:
        # choose next state using transition probabilities
        s1 = np.random.choice(states, p=P[s, a, :])

        # this is policy update part of algorithm 
        # epsilon-greedy policy update
        # choose action using udated epsilon-greedy policy
        # if this part is replaced and if we choose action using current policy
        # this whole algorithm becomes TD, neither SARSA nor Q-Learning
        if np.random.uniform(0.,1.) < epsilon:
            a1 = np.random.choice(actions)
        else:
            a1 = np.argmax(Q[s1, :])
            
        # this is policy estimate part of algorithm 
        # SARSA
        # Q[s, a] = Q[s, a] + alpha * (R[s, a] + gamma * Q[s1, a1] - Q[s, a])
        # Q-Learning
        Q[s, a] = Q[s, a] + alpha * (R[s, a] + gamma * max(Q[s1, :]) - Q[s, a])

        if (s1 == 3) or (s1 == 6):
            # if game is over, break while loop 
            done = True
        else:
            # if game is not over, continue playing game
            s = s1
            a = a1

print(Q)

[[ 0.23484059  0.85570446  0.31791229  0.23299121]
 [ 0.3332451   0.89128904  0.30141051  0.35832781]
 [ 0.59714572  0.92840636  0.67075245  0.50677206]
 [ 1.          1.          1.          1.        ]
 [ 0.25303943  0.18102757  0.82269687  0.21071712]
 [ 0.61471519 -0.54417305  0.5922635   0.32269063]
 [-1.         -1.         -1.         -1.        ]
 [ 0.05645621  0.10405808  0.76456092  0.08694896]
 [ 0.14149363  0.53391523  0.114884    0.12598085]
 [ 0.24382018  0.21406948  0.54586283  0.20021475]
 [ 0.32518441  0.00678814 -0.18524547  0.03145876]]
